# Анализ эксперимента run-pid-v2

Анализ работы инкрементального ПИД-регулятора на реальной установке. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from nn_laser_stabilizer.config.config import load_config
from nn_laser_stabilizer.paths import get_experiment_dir

## Загрузка эксперимента

In [ ]:
EXPERIMENT_NAME = "run-pid-v2"
EXPERIMENT_DATE = "2026-02-09"
EXPERIMENT_TIME = "14-03-13"

EXPERIMENT_DIR = get_experiment_dir(
    experiment_name=EXPERIMENT_NAME, 
    experiment_date=EXPERIMENT_DATE, 
    experiment_time=EXPERIMENT_TIME)

print(f"Директория эксперимента: {EXPERIMENT_DIR}")
print(f"Существует: {EXPERIMENT_DIR.exists()}")


In [ ]:
config = load_config(EXPERIMENT_DIR / "config.yaml")
pid_config = config.pid

print(f"Эксперимент: {config.experiment_name}")
print(f"\nПараметры PID:")
print(f"  kp = {pid_config.kp}")
print(f"  ki = {pid_config.ki}")
print(f"  kd = {pid_config.kd}")
print(f"  dt = {pid_config.dt}")
print(f"\nУставка: {config.setpoint}")
print(f"Warmup: {config.warmup_steps} шагов, output={config.warmup_output}")

## Парсинг логов

In [ ]:
import json

log_path = EXPERIMENT_DIR / "pid_data.jsonl"

records = []
with open(log_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            records.append(json.loads(line))

df = pd.DataFrame(records)

df["setpoint"] = config.setpoint
df["error"] = df["process_variable"] - df["setpoint"]

print(f"Загружено {len(df)} шагов")
df.head(10)

## Обзор данных

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df["step"], df["process_variable"], label="Process Variable", alpha=0.8, linewidth=0.5)
ax.axhline(config.setpoint, color="red", linestyle="--", label=f"Setpoint = {config.setpoint}")
ax.set_ylabel("Process Variable")
ax.set_xlabel("Step")
ax.legend(loc="upper right")
ax.set_title("Process Variable vs Setpoint")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df["step"], df["control_output"], label="Control Output", alpha=0.8, linewidth=0.5, color="green")
ax.set_ylabel("Control Output")
ax.set_xlabel("Step")
ax.legend(loc="upper right")
ax.set_title("Control Output")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df["step"], df["error"], label="Error", alpha=0.8, linewidth=0.5, color="purple")
ax.axhline(0, color="red", linestyle="--", alpha=0.5)
ax.set_ylabel("Error (PV - SP)")
ax.set_xlabel("Step")
ax.legend(loc="upper right")
ax.set_title("Error")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()